# ! See also TIL/geo/src/merge_dem_fr_it.sh $

##Setup
This should probably go to another file, to include with `%run Setup.ipynb`

# France

In [ ]:
%%bash

cd $download
!wget -i ign-rge-alti-5m-urls.txt
!wget --wait=20 --random-wait --no-http-keep-alive --limit-rate=300k -nc -i piemonte-alpi-dtm5-urls.txt
!wget --wait=20 --random-wait --no-http-keep-alive --limit-rate=300k -nc -i aosta-dtm0508-urls.txt

for f in RGE*29.7z; do 7z x $f; done
cd $workdir

In [ ]:
!gdalbuildvrt -a_srs EPSG:2154 -srcnodata -99999 -vrtnodata -99999 ignalps_dem.vrt \
  $download/RGEALTI_2-0_5M_ASC_LAMB93-IGN69_*/RGEALTI/1_DONNEES_LIVRAISON_*/RGEALTI_MNT_5M_ASC_LAMB93_IGN69_*/RGEALTI_FXX_*
!time gdaldem slope $=g_zstd $=g_tile ignalps_dem.vrt ignalps-lamb-slope.tif
!time gdaldem color-relief $=g_zstd $=g_tile -alpha ignalps-lamb-slope.tif ~/code/eddy-geek/TIL/geo/data/gdaldem-slope-oslo.clr ignalps-lamb-oslo.tif

In [ ]:
ln -s $download aoste_dtm_zips
rm -f aoste_dtm_zips.txt ; touch aoste_dtm_zips.txt ; for f in DTM0508_002_000{001..914} ; do echo "/vsizip/{aoste_dtm_zips/$f.zip}/$f.ASC" >> aoste_dtm_zips.txt ; done

gdalbuildvrt -a_srs EPSG:23032 -srcnodata -9999 -vrtnodata -9999 aoste-dem.vrt -input_file_list aoste_dtm_zips.txt
time gdaldem slope $=g_zstd aoste-dem.vrt aoste-utm32n-slope.tif
time gdaldem color-relief $=g_zstd -alpha aoste-utm32n-slope.tif gdaldem-slope-oslo.clr aoste-utm32n-oslo.tif

# Switzerland

This skips the selection of tiles to download. For a more reproducible download, check [Central-Alps.ipynb](./Central-Alps.ipynb)

In [ ]:
mkdir ch_downloads
cd ch_downloads
wget --no-http-keep-alive -nc -i ../ch.swisstopo.swissalti3d-valais-w2550-e2623-s1075-n1130.csv
cat ../ch.swisstopo.swissalti3d-z9valais-w2484-e2623-s1075-n1157.csv | rev | cut -d / -f1 | rev > _vrt_input.txt
gdalbuildvrt -input_file_list _vrt_input.txt z9valais-dem.vrt
cd ..
time gdaldem slope $=g_zstd ch_downloads/z9valais-dem.vrt valais-lv95-slope.tif

In [ ]:
resolution=2.388657133911758
extent='5.6250 43.5804 7.7343 46.558'
time gdalwarp $=g_tile \
  -co bigtiff=yes -co compress=zstd -co zstd_level=1 -co sparse_ok=true \
  -dstnodata 255 \
  -t_srs EPSG:3857 -tr $=resolution -$=resolution \
  -te_srs WGS84 -te $=extent \
  fr/ignalps-lamb-slope.tif it/piemont-utm32n-slope.tif aoste/aoste-utm32n-slope.tif \
  alex/ignalex-lamb-slope.tif ch/valais-lv95-slope.tif \
  alps/slopes-Lausanne-Jouques-Sanremo-Zermatt.tif